In [1]:
from diffusers import WanPipeline
from diffusers.utils import export_to_video

/home/ubuntu/dev/wan-api/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "pretrained_models/Wan2.1-T2V-1.3B-Diffusers"

In [3]:
import torch

pipe = WanPipeline.from_pretrained(model_id, torch_dtype=torch.bfloat16)
pipe.to("cuda")

Loading pipeline components...: 100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


WanPipeline {
  "_class_name": "WanPipeline",
  "_diffusers_version": "0.33.1",
  "_name_or_path": "pretrained_models/Wan2.1-T2V-1.3B-Diffusers",
  "scheduler": [
    "diffusers",
    "UniPCMultistepScheduler"
  ],
  "text_encoder": [
    "transformers",
    "UMT5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "T5TokenizerFast"
  ],
  "transformer": [
    "diffusers",
    "WanTransformer3DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKLWan"
  ]
}

In [4]:
prompt = "A cat and a dog baking a cake together in a kitchen. The cat is carefully measuring flour, while the dog is stirring the batter with a wooden spoon. The kitchen is cozy, with sunlight streaming through the window."
negative_prompt = "Bright tones, overexposed, static, blurred details, subtitles, style, works, paintings, images, static, overall gray, worst quality, low quality, JPEG compression residue, ugly, incomplete, extra fingers, poorly drawn hands, poorly drawn faces, deformed, disfigured, misshapen limbs, fused fingers, still picture, messy background, three legs, many people in the background, walking backwards"
num_frames = 33

In [5]:
frames = pipe(prompt=prompt, negative_prompt=negative_prompt, num_frames=num_frames).frames[0]
export_to_video(frames, "wan-t2v.mp4", fps=16)

100%|██████████| 50/50 [01:21<00:00,  1.62s/it]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'wan-t2v.mp4'

In [6]:
from IPython.display import Video

Video("wan-t2v.mp4")